In [2]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras import Model

In [12]:
train_path = './jhu_crowd/train/images/'
train_txt = './jhu_crowd/train/image_labels.txt'
x_train_savepath = './image_label/x_train.npy'
y_train_savepath = './image_label/y_train.npy'

test_path = './jhu_crowd/test/images/'
test_txt = './jhu_crowd/test/image_labels.txt'
x_test_savepath = './image_label/x_test.npy'
y_test_savepath = './image_label/y_test.npy'

In [7]:
def generateds(path, txt):
    f = open(txt, 'r')
    contents = f.readlines()
    f.close()
    x, y_ = [], []
    for content in contents:
        value = content.split(',')
        img_path = path + value[0] + '.jpg'
        img = Image.open(img_path)
        img = img.resize((96, 96))
        img = np.reshape(np.array(img.convert('L')), (96, 96, 1))
        img = img / 255.
        x.append(img)
        y_.append(value[1])

    x = np.array(x)
    y_ = np.array(y_)
    y_ = y_.astype(np.int64)
    return x, y_

In [13]:
if os.path.exists(x_train_savepath) and os.path.exists(y_train_savepath) and os.path.exists(
        x_test_savepath) and os.path.exists(y_test_savepath):
    print('-------------Load Datasets-----------------')
    x_train_save = np.load(x_train_savepath)
    y_train = np.load(y_train_savepath)
    x_test_save = np.load(x_test_savepath)
    y_test = np.load(y_test_savepath)
    x_train = np.reshape(x_train_save, (len(x_train_save), 96, 96, 1))
    x_test = np.reshape(x_test_save, (len(x_test_save), 96, 96, 1))
else:
    print('-------------Generate Datasets-----------------')
    x_train, y_train = generateds(train_path, train_txt)
    x_test, y_test = generateds(test_path, test_txt)

    print('-------------Save Datasets-----------------')
    x_train_save = np.reshape(x_train, (len(x_train), -1))
    x_test_save = np.reshape(x_test, (len(x_test), -1))
    np.save(x_train_savepath, x_train_save)
    np.save(y_train_savepath, y_train)
    np.save(x_test_savepath, x_test_save)
    np.save(y_test_savepath, y_test)

-------------Load Datasets-----------------


In [4]:
class ResnetBlock(Model):

    def __init__(self, filters, strides=1, residual_path=False):
        super(ResnetBlock, self).__init__()
        self.filters = filters
        self.strides = strides
        self.residual_path = residual_path

        self.c1 = Conv2D(filters, (3, 3), strides=strides, padding='same', use_bias=False)
        self.b1 = BatchNormalization()
        self.a1 = Activation('relu')

        self.c2 = Conv2D(filters, (3, 3), strides=1, padding='same', use_bias=False)
        self.b2 = BatchNormalization()

        if residual_path:
            self.down_c1 = Conv2D(filters, (1, 1), strides=strides, padding='same', use_bias=False)
            self.down_b1 = BatchNormalization()
        
        self.a2 = Activation('relu')

    def call(self, inputs):
        residual = inputs
        x = self.c1(inputs)
        x = self.b1(x)
        x = self.a1(x)

        x = self.c2(x)
        y = self.b2(x)

        if self.residual_path:
            residual = self.down_c1(inputs)
            residual = self.down_b1(residual)

        out = self.a2(y + residual)
        return out


class ResNet18(Model):

    def __init__(self, block_list, initial_filters=64):
        super(ResNet18, self).__init__()
        self.num_blocks = len(block_list)
        self.block_list = block_list
        self.out_filters = initial_filters
        self.c1 = Conv2D(self.out_filters, (3, 3), strides=1, padding='same', use_bias=False)
        self.b1 = BatchNormalization()
        self.a1 = Activation('relu')
        self.blocks = tf.keras.models.Sequential()

        for block_id in range(len(block_list)):
            for layer_id in range(block_list[block_id]):

                if block_id != 0 and layer_id == 0:
                    block = ResnetBlock(self.out_filters, strides=2, residual_path=True)
                else:
                    block = ResnetBlock(self.out_filters, residual_path=False)
                self.blocks.add(block)
            self.out_filters *= 2
        self.p1 = tf.keras.layers.GlobalAveragePooling2D()
        self.f1 = tf.keras.layers.Dense(1, activation='linear', kernel_regularizer=tf.keras.regularizers.l2())

    def call(self, inputs):
        x = self.c1(inputs)
        x = self.b1(x)
        x = self.a1(x)
        x = self.blocks(x)
        x = self.p1(x)
        y = self.f1(x)
        return y

In [26]:
model = ResNet18([2, 2, 2, 2])

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

In [27]:
checkpoint_save_path = "./checkpoint/ResNet18.ckpt"
if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)

-------------load the model-----------------


In [17]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                 save_weights_only=True,
                                                 save_best_only=True)

history = model.fit(x_train, y_train, batch_size=32, epochs=50, validation_data=(x_test, y_test), validation_freq=1,
                    callbacks=[cp_callback])

Epoch 1/50
71/71 [==============================] - 216s 3s/step - loss: 1375127.7500 - mae: 355.4348 - val_loss: 613602.4375 - val_mae: 361.8345
Epoch 2/50
71/71 [==============================] - 219s 3s/step - loss: 1319062.7500 - mae: 337.1463 - val_loss: 18329252.0000 - val_mae: 4010.7681
Epoch 3/50
71/71 [==============================] - 217s 3s/step - loss: 1257111.5000 - mae: 325.1395 - val_loss: 1745324.0000 - val_mae: 918.4911
Epoch 4/50
71/71 [==============================] - 221s 3s/step - loss: 1187956.1250 - mae: 314.8762 - val_loss: 480407.1562 - val_mae: 306.6506
Epoch 5/50
71/71 [==============================] - 218s 3s/step - loss: 1103076.2500 - mae: 306.7122 - val_loss: 472907.2500 - val_mae: 331.3762
Epoch 6/50
71/71 [==============================] - 221s 3s/step - loss: 1092034.1250 - mae: 317.8212 - val_loss: 6919977.0000 - val_mae: 1927.7661
Epoch 7/50
71/71 [==============================] - 220s 3s/step - loss: 1037206.0000 - mae: 312.1949 - val_loss: 2324

In [18]:
model.summary()

Model: "res_net18_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          multiple                  576       
                                                                 
 batch_normalization_20 (Bat  multiple                 256       
 chNormalization)                                                
                                                                 
 activation_17 (Activation)  multiple                  0         
                                                                 
 sequential_1 (Sequential)   (32, 12, 12, 512)         11176448  
                                                                 
 global_average_pooling2d_1   multiple                 0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_1 (Dense)             multiple                  

In [19]:
file = open('./weights.txt', 'w')
for v in model.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')
file.close()

In [29]:
test_img = []

f = open('./jhu_crowd/test/image_labels.txt', 'r')
contents = f.readlines()
f.close()

for content in contents:
    value = content.split(',')
    test_img.append(value[0] + '.jpg')

test_prediction = model.predict(x_test)

50/50 [==============================] - 30s 599ms/step


In [56]:
import pandas as pd

df = pd.DataFrame([test_img, np.reshape(test_prediction, (len(test_prediction)))]).T
csv_file_path = './prediction.csv'
df.to_csv(csv_file_path, index=False)

In [57]:
!python ./eval_tool/jhucrowd_eval.py jhu_crowd prediction.csv test

                     mae_low , mse_low , mae_med , mse_med , mae_high , mse_high , mae_weather , mse_weather , mae_overall , mse_overall
prediction           ,   141.4 ,   223.8 ,   138.5 ,   240.2 ,   931.2 ,  1512.4 ,        349.5 ,        842.0 ,      257.2 ,      622.4
